# Mosaic Tracker - Single Particle Tracking Tutorial

This notebook demonstrates how to use the `mosaic_tracker` package for single particle tracking and trajectory analysis.

## Contents
1. Installation and Setup
2. Creating Synthetic Data
3. Particle Detection
4. Trajectory Linking
5. MSD and MSS Analysis
6. Visualization
7. Working with Real Data

## 1. Installation and Setup

In [ ]:
# Install the package (run once)
# !pip install -e /path/to/mosaic_tracker[all]

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from mosaic_tracker import (
    ParticleTracker,
    TrackerParameters,
    create_synthetic_movie,
    plot_trajectories,
    plot_trajectory,
    plot_msd,
    plot_mss,
    plot_diffusion_histogram,
    plot_frame_with_particles
)

%matplotlib inline
plt.rcParams['figure.figsize'] = [10, 8]
plt.rcParams['figure.dpi'] = 100

## 2. Creating Synthetic Data

First, let's create a synthetic movie with diffusing particles to test the tracking.

In [ ]:
# Create synthetic movie with known diffusion coefficient
movie = create_synthetic_movie(
    n_frames=200,
    size=(256, 256),
    n_particles=20,
    diffusion_coeff=0.5,  # pixels^2/frame
    snr=8.0,
    particle_sigma=2.0,
    background=100.0,
    seed=42
)

print(f"Movie shape: {movie.shape}")
print(f"Intensity range: {movie.min():.1f} - {movie.max():.1f}")

In [ ]:
# Visualize a few frames
fig, axes = plt.subplots(1, 4, figsize=(16, 4))
for i, ax in enumerate(axes):
    frame_idx = i * 50
    ax.imshow(movie[frame_idx], cmap='gray')
    ax.set_title(f'Frame {frame_idx}')
    ax.axis('off')
plt.tight_layout()
plt.show()

## 3. Particle Detection and Tracking

Now let's configure the tracker and run detection.

In [ ]:
# Configure tracking parameters
params = TrackerParameters(
    # Detection parameters
    radius=3,              # Particle radius (pixels)
    percentile=0.5,        # Intensity percentile (lower = more stringent)
    cutoff_score=0.0,      # Non-particle discrimination (0 = disabled)
    
    # Linking parameters
    max_displacement=10.0, # Maximum displacement per frame (pixels)
    link_range=2,          # Frames to look ahead for gap closing
    min_length=20,         # Minimum trajectory length to keep
    
    # Physical parameters (for your specific microscope setup)
    pixel_size=0.080,      # µm per pixel (e.g., 80 nm for 100x objective)
    frame_interval=0.033   # seconds between frames (e.g., 30 fps)
)

# Create tracker
tracker = ParticleTracker(params)

# Load movie
tracker.load_movie(movie)
print(tracker)

In [ ]:
# Detect particles
tracker.detect_particles()

print(f"Total particles detected: {tracker.n_particles}")
print(f"Mean particles per frame: {tracker.n_particles / tracker.n_frames:.1f}")

In [ ]:
# Visualize detection results
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Frame 0
plot_frame_with_particles(movie, tracker.particles, frame_idx=0, ax=axes[0])
axes[0].set_title('Frame 0 with detected particles')

# Frame 100
plot_frame_with_particles(movie, tracker.particles, frame_idx=100, ax=axes[1])
axes[1].set_title('Frame 100 with detected particles')

plt.tight_layout()
plt.show()

## 4. Trajectory Linking

In [ ]:
# Link particles into trajectories
tracker.link_trajectories()

print(f"Number of trajectories: {tracker.n_trajectories}")
print(f"Trajectory lengths: {[t.length for t in tracker.trajectories]}")

In [ ]:
# Visualize all trajectories
fig, ax = plt.subplots(figsize=(10, 10))
plot_trajectories(
    tracker.trajectories,
    ax=ax,
    color_by='id',
    alpha=0.7,
    linewidth=1.5,
    background=movie[0]
)
plt.title('All Tracked Trajectories')
plt.show()

In [ ]:
# Visualize a single trajectory with time coloring
if tracker.trajectories:
    # Get the longest trajectory
    longest_traj = max(tracker.trajectories, key=lambda t: t.length)
    
    fig, ax = plt.subplots(figsize=(8, 8))
    plot_trajectory(longest_traj, ax=ax, color_by_time=True)
    plt.title(f'Trajectory {longest_traj.id} (length={longest_traj.length})')
    plt.show()

## 5. MSD and MSS Analysis

In [ ]:
# Run analysis on all trajectories
results = tracker.analyze()

print(f"Analyzed {len(results)} trajectories")
print(tracker.summary())

In [ ]:
# Get summary as DataFrame
df = tracker.get_summary_dataframe()
df.head(10)

In [ ]:
# Plot MSD for a specific trajectory
if tracker.trajectories:
    traj_id = tracker.trajectories[0].id
    msd = tracker.get_msd(traj_id)
    
    fig, ax = plt.subplots(figsize=(8, 6))
    plot_msd(msd, ax=ax)
    plt.title(f'MSD for Trajectory {traj_id}')
    plt.show()
    
    print(f"Diffusion coefficient: {msd.diffusion_coeff:.4f} µm²/s")
    print(f"Alpha (anomalous exponent): {msd.slope:.3f}")
    print(f"Motion type: {msd.motion_type}")

In [ ]:
# Plot MSS for a trajectory
if tracker.trajectories:
    mss = tracker.get_mss(traj_id)
    
    fig, ax = plt.subplots(figsize=(8, 6))
    plot_mss(mss, ax=ax)
    plt.title(f'MSS for Trajectory {traj_id}')
    plt.show()
    
    print(f"MSS slope: {mss.slope:.3f}")
    print(f"Motion type: {mss.motion_type}")
    print(f"Self-similar: {mss.is_self_similar}")

## 6. Population Analysis

In [ ]:
# Distribution of diffusion coefficients
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

plot_diffusion_histogram(results, ax=axes[0], log_scale=True)

# Alpha distribution
alphas = [r['alpha'] for r in results]
axes[1].hist(alphas, bins=20, color='steelblue', edgecolor='black')
axes[1].axvline(1.0, color='red', linestyle='--', label='Normal diffusion (α=1)')
axes[1].set_xlabel('Alpha (anomalous exponent)')
axes[1].set_ylabel('Count')
axes[1].set_title('Distribution of Anomalous Exponents')
axes[1].legend()

plt.tight_layout()
plt.show()

In [ ]:
# Motion type classification
from mosaic_tracker import plot_motion_type_pie

fig, ax = plt.subplots(figsize=(8, 8))
plot_motion_type_pie(results, ax=ax)
plt.show()

In [ ]:
# Scatter plot of diffusion coefficient vs trajectory length
fig, ax = plt.subplots(figsize=(10, 6))

lengths = [r['length'] for r in results]
d_coeffs = [r['diffusion_coefficient'] for r in results]
motion_types = [r['motion_type'] for r in results]

# Color by motion type
colors = {'normal_diffusion': 'blue', 'subdiffusive/confined': 'red', 
          'superdiffusive': 'green', 'ballistic/directed': 'orange',
          'stationary': 'gray'}
c = [colors.get(mt, 'black') for mt in motion_types]

ax.scatter(lengths, d_coeffs, c=c, alpha=0.6)
ax.set_xlabel('Trajectory Length (frames)')
ax.set_ylabel('Diffusion Coefficient (µm²/s)')
ax.set_title('Diffusion Coefficient vs Trajectory Length')
ax.set_yscale('log')

# Add legend
for mt, color in colors.items():
    ax.scatter([], [], c=color, label=mt)
ax.legend()

plt.show()

## 7. Export Results

In [ ]:
# Export trajectory positions
positions_df = tracker.get_positions_dataframe()
positions_df.head()

In [ ]:
# Save to CSV
# positions_df.to_csv('trajectory_positions.csv', index=False)
# df.to_csv('trajectory_summary.csv', index=False)

## 8. Working with Real Data

For your 40nm GEM tracking, use these recommended parameters:

In [ ]:
# Example for 40nm GEM tracking
gem_params = TrackerParameters(
    radius=2,              # GEMs appear as small spots (~2-3 pixels)
    percentile=0.1,        # Lower = more stringent detection
    cutoff_score=0.0,      # Disable for homogeneous particles
    max_displacement=10,   # Adjust based on expected diffusion
    link_range=2,          # Allow 1-frame gaps
    min_length=30,         # Require longer tracks for reliable MSD
    pixel_size=0.080,      # 80 nm per pixel (adjust for your setup)
    frame_interval=0.033   # 33 ms (30 fps)
)

print("Parameters for 40nm GEM tracking:")
print(gem_params)

In [ ]:
# Load and track your real data
# tracker = ParticleTracker(gem_params)
# tracker.load_movie('/path/to/your/movie.tif')
# tracker.run()  # Runs detect -> link -> analyze
# print(tracker.summary())

## Tips for Parameter Tuning

1. **radius**: Should be slightly larger than the apparent particle radius in pixels
2. **percentile**: Start with 1.0 and decrease if detecting too many false positives
3. **max_displacement**: Should be at least 2-3x the expected displacement per frame
4. **link_range**: Increase if particles blink (e.g., quantum dots)
5. **min_length**: Longer tracks give more reliable MSD estimates